<a href="https://colab.research.google.com/github/diaboloshogunate/GoogleColabML/blob/main/MNIST-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [139]:
# Imports
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

from copy import deepcopy
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms

from torchvision.transforms import ToTensor
import copy

In [140]:
# Enable GPU if avialable
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [141]:
def build_resnet_model():
  model = models.resnet50(pretrained=True)
  model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

  for param in model.parameters():
    param.requires_grad = False
  num_ftrs = model.fc.in_features
  model.fc = nn.Linear(num_ftrs, 10)

  return model

In [142]:
def build_vgg_model():
  model = models.vgg19(pretrained=True)
  model.features[0] = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

  for param in model.parameters():
    param.requires_grad = False
  num_ftrs = model.fc.in_features
  model.fc = nn.Linear(num_ftrs, 10)

  return model

In [143]:
class SubsampleDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        return x, y

    def __len__(self):
        return len(self.data)

In [144]:
def get_trainloader(sample_size):
  train_transformer = transforms.Compose([transforms.Resize(224),transforms.ToTensor()])
  train_set = torchvision.datasets.MNIST('./data', download=True, train=True, transform=train_transformer)

  counter = np.zeros(10)
  train_set_subsample = SubsampleDataset([], [])

  for data in train_set:
    image = data[0]
    label = data[1]

    if counter[label] <= sample_size:
        train_set_subsample.data.append(deepcopy(image))
        train_set_subsample.targets.append(deepcopy(label))
        counter[label] += 1

    if sum(counter) >= sample_size*10:
        break

  return torch.utils.data.DataLoader(train_set_subsample, batch_size=batch_size, shuffle=True, num_workers=2)


In [145]:
def get_testloader():
  test_transformer = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
  ])
  test_set = torchvision.datasets.MNIST('./data', download=True, train=False, transform=test_transformer)
  return torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

In [146]:
# train model
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"Train Loss: {loss:>7f}")

In [147]:
# test
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [148]:
# parameters
epochs = 30
batch_size = 100
sizes = [10, 20, 30, 40, 50, 60, 70, 80, 90]
learning_rate=0.01
momentum=0.9

In [149]:
# resnet experiment
for sample_size in sizes:
  model = build_resnet_model()
  model.to(device)
  optimizer=optim.SGD(model.fc.parameters(), lr=learning_rate, momentum=momentum)
  loss_function = nn.CrossEntropyLoss()
  trainloader = get_trainloader(sample_size)
  testloader = get_testloader()
  for t in range(epochs):
    print(f"Model: Resnet, Size: {sample_size}, Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_function, optimizer)
    test(testloader, model, loss_function)

Model: Resnet, Size: 10, Epoch 1
-------------------------------
Train Loss: 2.355883
Test Error: Accuracy: 5.1%, Avg loss: 2.331955 

Model: Resnet, Size: 10, Epoch 2
-------------------------------
Train Loss: 2.275129
Test Error: Accuracy: 9.9%, Avg loss: 2.320814 

Model: Resnet, Size: 10, Epoch 3
-------------------------------
Train Loss: 2.195318
Test Error: Accuracy: 12.9%, Avg loss: 2.332299 

Model: Resnet, Size: 10, Epoch 4
-------------------------------
Train Loss: 2.128587
Test Error: Accuracy: 9.1%, Avg loss: 2.349047 

Model: Resnet, Size: 10, Epoch 5
-------------------------------
Train Loss: 2.032854
Test Error: Accuracy: 8.6%, Avg loss: 2.349093 

Model: Resnet, Size: 10, Epoch 6
-------------------------------
Train Loss: 1.891156
Test Error: Accuracy: 11.8%, Avg loss: 2.338166 

Model: Resnet, Size: 10, Epoch 7
-------------------------------
Train Loss: 1.732244
Test Error: Accuracy: 13.3%, Avg loss: 2.331474 

Model: Resnet, Size: 10, Epoch 8
-------------------

RuntimeError: ignored

In [ ]:
# vgg experiment
for sample_size in sizes:
  model = build_vgg_model()
  model.to(device)
  optimizer=optim.SGD(model.fc.parameters(), lr=learning_rate, momentum=momentum)
  loss_function = nn.CrossEntropyLoss()
  trainloader = get_trainloader(sample_size)
  testloader = get_testloader()
  for t in range(epochs):
    print(f"Model: Resnet, Size: {sample_size}, Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_function, optimizer)
    test(testloader, model, loss_function)